In [91]:
%matplotlib inline

import sys
from pathlib import Path

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

SCRIPT_DIR = Path.cwd()
sys.path.append(str(SCRIPT_DIR.parent))

from util_data.dataIterator import DataIterator
tf.compat.v1.disable_eager_execution() 

Loading the processed data

In [75]:
training_data = np.load(open("training_data/mushrooms_training_data.npy", 'rb'))
training_labels = np.load(open("training_data/mushrooms_training_labels.npy", 'rb'))
validation_data = np.load(open("training_data/mushrooms_validation_data.npy", 'rb'))
validation_labels = np.load(open("training_data/mushrooms_validation_labels.npy", 'rb'))
test_data = np.load(open("training_data/mushrooms_test_data.npy", 'rb'))
test_labels = np.load(open("training_data/mushrooms_test_labels.npy", 'rb'))

print data types and shapes

In [76]:
print("Loaded data type = {}".format(type(training_data)))
print("Training data shape = {}".format(training_data.shape))
print("Training labels shape = {}".format(training_labels.shape))
print("Validation data shape = {}".format(validation_data.shape))
print("Validation labels shape = {}".format(validation_labels.shape))
print("Test data shape = {}".format(test_data.shape))
print("Test labels shape = {}".format(test_labels.shape))

Loaded data type = <class 'numpy.ndarray'>
Training data shape = (6500, 117)
Training labels shape = (6500, 1)
Validation data shape = (812, 117)
Validation labels shape = (812, 1)
Test data shape = (812, 117)
Test labels shape = (812, 1)


In [126]:
#parameters

fully_connected_layers=3
fully_connected_size=50
fully_connected_activationfunction=tf.nn.relu

In [159]:
#placeholders data input

# Data inputs
x = tf.Variable(initial_value=tf.constant([], dtype=tf.float32), trainable=False, name="inputs")
print(x)

# Labels inputs
y = tf.Variable(initial_value=tf.constant([], dtype=tf.float32), trainable=False, name="labels")
print(y)

# Learning rate input
learning_rate_input = tf.Variable(initial_value=tf.constant(0.0, dtype=tf.float32), trainable=False, name="learning_rate")
print(learning_rate_input)

# Used in training to keep track of variables updates
global_step = tf.Variable(initial_value=0, trainable=False, dtype=tf.int32, name="global_step")
print(global_step)


<tf.Variable 'inputs_41:0' shape=(0,) dtype=float32>
<tf.Variable 'labels_39:0' shape=(0,) dtype=float32>
<tf.Variable 'learning_rate_39:0' shape=() dtype=float32>
<tf.Variable 'global_step_1:0' shape=() dtype=int32>


In [139]:
print(global_step)

<tf.Variable 'hj:0' shape=() dtype=int32>


In [137]:
#creating hidden layers for the model

dense_layers=fully_connected_layers

dense = tf.keras.layers.Dense(units=fully_connected_size,
                              activation=fully_connected_activationfunction,
                              name="dense_1")(x)
print(dense)

dense_layers -=1
dense_layer_count= 2

for i in range(dense_layers):
    # Shape: (batch_size, fully_connected_size)
    dense = tf.keras.layers.Dense(units=fully_connected_size,
                                  activation=fully_connected_activationfunction,
                                  name="dense_{}".format(dense_layer_count))(dense)

    print(dense)

    dense_layer_count += 1


Tensor("dense_1_5/Relu:0", shape=(None, 50), dtype=float32)
Tensor("dense_2_2/Relu:0", shape=(None, 50), dtype=float32)
Tensor("dense_3_2/Relu:0", shape=(None, 50), dtype=float32)


Output Layer

In [142]:
# Shape: (batch_size, 1)
output_layer = tf.keras.layers.Dense( 
                               units=1, 
                               activation=tf.nn.sigmoid,
                               name="output")(dense)

print(output_layer)

Tensor("output/Sigmoid:0", shape=(None, 1), dtype=float32)


In [153]:
#cost optimzer

with tf.name_scope("cost_and_optimizer"):
    
    # Loss value for the model output of each data point fed to the graph
    # Shape: (batch_size, 1)
    loss_op = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,
                                                      logits=output_layer,
                                                      name="loss_op")
    print(loss_op)
    
    # Cost value for all the data points fed to the model
    # Shape: ()
    cost = tf.reduce_sum(loss_op, name="cost")
    print(cost)
    
    # Model optimizer
    optimizer = tf.optimizers.SGD(learning_rate=learning_rate,
                                                 name="gradient_descent_optimizer")
    
    # Trining operation. Perform one parameters update. Increment global_step by 1.
    training_op = optimizer.minimize(cost, global_step=global_step)

Tensor("cost_and_optimizer_7/loss_op:0", shape=(None, 1), dtype=float32)
Tensor("cost_and_optimizer_7/cost:0", shape=(), dtype=float32)


TypeError: minimize() got an unexpected keyword argument 'global_step'